In [8]:
# import os
# import random
# import shutil
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
# import torchvision.transforms as transforms
# from torchvision.models import resnet18, ResNet18_Weights
# from PIL import Image
# import librosa
# import numpy as np
# from collections import defaultdict
# import json

# # =========================
# # 0. 설정
# # =========================
# SEED = 42
# random.seed(SEED)
# torch.manual_seed(SEED)
# np.random.seed(SEED)

# # 경로
# BEHAVIOR_ROOT = "files/1_Animal_Behavior"
# EMOTION_ROOT = "files/2_Animal_emotions"
# SOUND_ROOT = "files/3_Animal_Sound"
# WORK_DIR = "files/work/omni_dataset"

# # 하이퍼파라미터
# BATCH_SIZE = 16
# EPOCHS = 100
# LR_VIDEO = 1e-4
# LR_AUDIO = 1e-5
# DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
# NUM_WORKERS = 4
# SR = 16000
# MAX_AUDIO_LEN = SR * 5

# # Loss Weighting
# LOSS_WEIGHTS = {
#     'behavior': 1.0,
#     'emotion': 0.8,
#     'sound': 0.6
# }

# print(f"🎯 Device: {DEVICE}, Workers: {NUM_WORKERS}")

# # Audio Feature Extractor
# AUDIO_MODEL_NAME = "facebook/wav2vec2-base"
# FEATURE_EXTRACTOR = Wav2Vec2FeatureExtractor.from_pretrained(AUDIO_MODEL_NAME)

# # =========================
# # 1. 데이터셋 준비 (기존 동일)
# # =========================
# def prepare_omni_dataset():
#     """behavior + emotion 이미지, sound 오디오를 통합 디렉토리로 구성"""
#     if os.path.exists(WORK_DIR):
#         shutil.rmtree(WORK_DIR)
    
#     for split in ['train', 'val', 'test']:
#         os.makedirs(os.path.join(WORK_DIR, split, 'behavior'), exist_ok=True)
#         os.makedirs(os.path.join(WORK_DIR, split, 'emotion'), exist_ok=True)
#         os.makedirs(os.path.join(WORK_DIR, split, 'sound'), exist_ok=True)
    
#     print("📦 Collecting behavior data...")
#     behavior_samples = collect_samples(BEHAVIOR_ROOT, exts=['.jpg', '.png', '.jpeg'])
    
#     print("📦 Collecting emotion data...")
#     emotion_samples = collect_samples(EMOTION_ROOT, exts=['.jpg', '.png', '.jpeg'])
    
#     print("📦 Collecting sound data...")
#     sound_samples = collect_samples(SOUND_ROOT, exts=['.wav', '.mp3', '.m4a'])
    
#     # 8:1:1 split
#     split_and_copy(behavior_samples, 'behavior', img=True)
#     split_and_copy(emotion_samples, 'emotion', img=True)
#     split_and_copy(sound_samples, 'sound', img=False)
    
#     print("✅ Omni dataset ready!")

# def collect_samples(root, exts):
#     samples = []
#     for class_dir in os.listdir(root):
#         class_path = os.path.join(root, class_dir)
#         if not os.path.isdir(class_path):
#             continue
        
#         for filename in os.listdir(class_path):
#             if any(filename.lower().endswith(ext) for ext in exts):
#                 samples.append((class_dir, os.path.join(class_path, filename)))
    
#     print(f"  → {len(samples)} samples, {len(set(s[0] for s in samples))} classes")
#     return samples

# def split_and_copy(samples, task_name, img=True):
#     random.shuffle(samples)
    
#     class_samples = defaultdict(list)
#     for label, path in samples:
#         class_samples[label].append(path)
    
#     train_files, val_files, test_files = [], [], []
#     for label, paths in class_samples.items():
#         n = len(paths)
#         n_train = int(n * 0.8)
#         n_val = int(n * 0.1)
        
#         train_files.extend([(label, p) for p in paths[:n_train]])
#         val_files.extend([(label, p) for p in paths[n_train:n_train+n_val]])
#         test_files.extend([(label, p) for p in paths[n_train+n_val:]])
    
#     for split_name, files in [('train', train_files), ('val', val_files), ('test', test_files)]:
#         print(f"  Copying {task_name}/{split_name}: {len(files)}")
#         for label, src in tqdm(files, desc=f"{task_name}/{split_name}"):
#             dst_dir = os.path.join(WORK_DIR, split_name, task_name, label)
#             os.makedirs(dst_dir, exist_ok=True)
#             shutil.copy(src, os.path.join(dst_dir, os.path.basename(src)))

# # =========================
# # 2. 태스크별 Dataset (분리)
# # =========================
# class ImageDataset(Dataset):
#     """Behavior / Emotion용 이미지 데이터셋"""
#     def __init__(self, task_dir, augment=False):
#         self.samples = []
#         self.label_to_id = {}
        
#         for label in sorted(os.listdir(task_dir)):
#             label_dir = os.path.join(task_dir, label)
#             if not os.path.isdir(label_dir):
#                 continue
            
#             if label not in self.label_to_id:
#                 self.label_to_id[label] = len(self.label_to_id)
            
#             for filename in os.listdir(label_dir):
#                 if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
#                     self.samples.append((os.path.join(label_dir, filename), label))
        
#         # 전처리
#         if augment:
#             self.transform = transforms.Compose([
#                 transforms.Resize((256, 256)),
#                 transforms.RandomCrop(224),
#                 transforms.RandomHorizontalFlip(),
#                 transforms.ColorJitter(0.2, 0.2, 0.2),
#                 transforms.ToTensor(),
#                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#             ])
#         else:
#             self.transform = transforms.Compose([
#                 transforms.Resize((224, 224)),
#                 transforms.ToTensor(),
#                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#             ])
        
#         print(f"  📊 {os.path.basename(task_dir)}: {len(self.samples)} samples, {len(self.label_to_id)} classes")
    
#     def __len__(self):
#         return len(self.samples)
    
#     def __getitem__(self, idx):
#         path, label = self.samples[idx]
#         img = Image.open(path).convert('RGB')
#         img_tensor = self.transform(img)
#         label_id = self.label_to_id[label]
#         return img_tensor, label_id

# class AudioDataset(Dataset):
#     """Sound용 오디오 데이터셋"""
#     def __init__(self, task_dir, augment=False):
#         self.samples = []
#         self.label_to_id = {}
#         self.augment = augment
        
#         for label in sorted(os.listdir(task_dir)):
#             label_dir = os.path.join(task_dir, label)
#             if not os.path.isdir(label_dir):
#                 continue
            
#             if label not in self.label_to_id:
#                 self.label_to_id[label] = len(self.label_to_id)
            
#             for filename in os.listdir(label_dir):
#                 if filename.lower().endswith(('.wav', '.mp3', '.m4a')):
#                     self.samples.append((os.path.join(label_dir, filename), label))
        
#         print(f"  📊 {os.path.basename(task_dir)}: {len(self.samples)} samples, {len(self.label_to_id)} classes")
    
#     def __len__(self):
#         return len(self.samples)
    
#     def __getitem__(self, idx):
#         path, label = self.samples[idx]
        
#         try:
#             waveform, _ = librosa.load(path, sr=SR, mono=True)
#         except:
#             waveform = np.zeros(MAX_AUDIO_LEN)
        
#         # 증강 (선택)
#         if self.augment and random.random() > 0.5:
#             # Pitch shift
#             n_steps = random.uniform(-2, 2)
#             waveform = librosa.effects.pitch_shift(waveform, sr=SR, n_steps=n_steps)
        
#         # Padding/Truncate
#         if len(waveform) > MAX_AUDIO_LEN:
#             waveform = waveform[:MAX_AUDIO_LEN]
#         else:
#             waveform = np.pad(waveform, (0, MAX_AUDIO_LEN - len(waveform)))
        
#         inputs = FEATURE_EXTRACTOR(waveform, sampling_rate=SR, return_tensors="pt")
#         audio_tensor = inputs.input_values.squeeze(0)
#         label_id = self.label_to_id[label]
        
#         return audio_tensor, label_id

# # =========================
# # 3. 모델 정의 (개선)
# # =========================
# class VideoMultiHeadModel(nn.Module):
#     """Vision Backbone + Behavior/Emotion Heads"""
#     def __init__(self, num_behavior_classes, num_emotion_classes):
#         super().__init__()
        
#         # Shared Backbone
#         weights = ResNet18_Weights.IMAGENET1K_V1
#         backbone = resnet18(weights=weights)
#         self.backbone = nn.Sequential(*list(backbone.children())[:-1])  # 512-dim
        
#         # Task Heads
#         self.behavior_head = nn.Linear(512, num_behavior_classes)
#         self.emotion_head = nn.Linear(512, num_emotion_classes)
    
#     def forward(self, x, task='behavior'):
#         """task별로 적절한 head 선택"""
#         feat = self.backbone(x).squeeze(-1).squeeze(-1)  # [B, 512]
        
#         if task == 'behavior':
#             return self.behavior_head(feat)
#         elif task == 'emotion':
#             return self.emotion_head(feat)
#         else:
#             raise ValueError(f"Unknown task: {task}")

# class AudioModel(nn.Module):
#     """독립 Audio 모델"""
#     def __init__(self, num_sound_classes):
#         super().__init__()
#         self.model = Wav2Vec2ForSequenceClassification.from_pretrained(
#             AUDIO_MODEL_NAME,
#             num_labels=num_sound_classes,
#             ignore_mismatched_sizes=True
#         )
    
#     def forward(self, audio):
#         return self.model(input_values=audio).logits

# # =========================
# # 4. Task Alternating 학습
# # =========================
# def train():
#     """Task별로 번갈아가며 학습"""
#     prepare_omni_dataset()
    
#     # 데이터셋 로드
#     print("\n🔄 Loading datasets...")
#     behavior_train = ImageDataset(os.path.join(WORK_DIR, 'train', 'behavior'), augment=True)
#     behavior_val = ImageDataset(os.path.join(WORK_DIR, 'val', 'behavior'), augment=False)
    
#     emotion_train = ImageDataset(os.path.join(WORK_DIR, 'train', 'emotion'), augment=True)
#     emotion_val = ImageDataset(os.path.join(WORK_DIR, 'val', 'emotion'), augment=False)
    
#     sound_train = AudioDataset(os.path.join(WORK_DIR, 'train', 'sound'), augment=True)
#     sound_val = AudioDataset(os.path.join(WORK_DIR, 'val', 'sound'), augment=False)
    
#     # DataLoader
#     behavior_train_loader = DataLoader(behavior_train, batch_size=BATCH_SIZE, shuffle=True, 
#                                       num_workers=NUM_WORKERS, pin_memory=True)
#     behavior_val_loader = DataLoader(behavior_val, batch_size=BATCH_SIZE, shuffle=False, 
#                                     num_workers=NUM_WORKERS//2, pin_memory=True)
    
#     emotion_train_loader = DataLoader(emotion_train, batch_size=BATCH_SIZE, shuffle=True, 
#                                      num_workers=NUM_WORKERS, pin_memory=True)
#     emotion_val_loader = DataLoader(emotion_val, batch_size=BATCH_SIZE, shuffle=False, 
#                                    num_workers=NUM_WORKERS//2, pin_memory=True)
    
#     sound_train_loader = DataLoader(sound_train, batch_size=BATCH_SIZE, shuffle=True, 
#                                    num_workers=2, pin_memory=True)
#     sound_val_loader = DataLoader(sound_val, batch_size=BATCH_SIZE, shuffle=False, 
#                                  num_workers=1, pin_memory=True)
    
#     # 모델 초기화
#     video_model = VideoMultiHeadModel(
#         num_behavior_classes=len(behavior_train.label_to_id),
#         num_emotion_classes=len(emotion_train.label_to_id)
#     ).to(DEVICE)
    
#     audio_model = AudioModel(
#         num_sound_classes=len(sound_train.label_to_id)
#     ).to(DEVICE)
    
#     # Optimizer (분리)
#     video_optimizer = torch.optim.AdamW(video_model.parameters(), lr=LR_VIDEO, weight_decay=0.01)
#     audio_optimizer = torch.optim.AdamW(audio_model.parameters(), lr=LR_AUDIO, weight_decay=0.01)
    
#     # Loss
#     criterion = nn.CrossEntropyLoss()
#     scaler = torch.amp.GradScaler('cuda')
    
#     # Task Iteration Tracker
#     task_iters = {
#         'behavior': iter(behavior_train_loader),
#         'emotion': iter(emotion_train_loader),
#         'sound': iter(sound_train_loader)
#     }
    
#     # 학습 루프
#     best_avg_acc = 0
#     history = []
    
#     # Epoch당 총 iteration (가장 긴 태스크 기준)
#     max_iters = max(len(behavior_train_loader), len(emotion_train_loader), len(sound_train_loader))
    
#     for epoch in range(EPOCHS):
#         video_model.train()
#         audio_model.train()
        
#         # Task별 loss 누적
#         task_losses = {'behavior': 0, 'emotion': 0, 'sound': 0}
#         task_counts = {'behavior': 0, 'emotion': 0, 'sound': 0}
        
#         pbar = tqdm(range(max_iters), desc=f"Epoch {epoch+1}/{EPOCHS}")
        
#         for step in pbar:
#             # Task 순환: behavior → emotion → sound
#             current_task = ['behavior', 'emotion', 'sound'][step % 3]
            
#             # 해당 task의 batch 가져오기
#             try:
#                 if current_task == 'behavior':
#                     imgs, labels = next(task_iters['behavior'])
#                 elif current_task == 'emotion':
#                     imgs, labels = next(task_iters['emotion'])
#                 else:  # sound
#                     audios, labels = next(task_iters['sound'])
#             except StopIteration:
#                 # Iterator 재생성
#                 if current_task == 'behavior':
#                     task_iters['behavior'] = iter(behavior_train_loader)
#                     imgs, labels = next(task_iters['behavior'])
#                 elif current_task == 'emotion':
#                     task_iters['emotion'] = iter(emotion_train_loader)
#                     imgs, labels = next(task_iters['emotion'])
#                 else:
#                     task_iters['sound'] = iter(sound_train_loader)
#                     audios, labels = next(task_iters['sound'])
            
#             # GPU 이동
#             labels = labels.to(DEVICE)
            
#             # Forward + Loss
#             with torch.amp.autocast('cuda'):
#                 if current_task in ['behavior', 'emotion']:
#                     imgs = imgs.to(DEVICE)
#                     logits = video_model(imgs, task=current_task)
#                     loss = criterion(logits, labels) * LOSS_WEIGHTS[current_task]
                    
#                     video_optimizer.zero_grad()
#                     scaler.scale(loss).backward()
#                     scaler.step(video_optimizer)
#                     scaler.update()
                    
#                 else:  # sound
#                     audios = audios.to(DEVICE)
#                     logits = audio_model(audios)
#                     loss = criterion(logits, labels) * LOSS_WEIGHTS['sound']
                    
#                     audio_optimizer.zero_grad()
#                     scaler.scale(loss).backward()
#                     scaler.step(audio_optimizer)
#                     scaler.update()
            
#             task_losses[current_task] += loss.item()
#             task_counts[current_task] += 1
            
#             pbar.set_postfix({
#                 'task': current_task,
#                 'loss': loss.item()
#             })
        
#         # Validation
#         print("\n🔍 Validation...")
#         video_model.eval()
#         audio_model.eval()
        
#         # Behavior Val
#         correct_b, total_b = 0, 0
#         with torch.no_grad():
#             for imgs, labels in behavior_val_loader:
#                 imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
#                 logits = video_model(imgs, task='behavior')
#                 pred = logits.argmax(-1)
#                 correct_b += (pred == labels).sum().item()
#                 total_b += labels.size(0)
#         acc_b = correct_b / total_b
        
#         # Emotion Val
#         correct_e, total_e = 0, 0
#         with torch.no_grad():
#             for imgs, labels in emotion_val_loader:
#                 imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
#                 logits = video_model(imgs, task='emotion')
#                 pred = logits.argmax(-1)
#                 correct_e += (pred == labels).sum().item()
#                 total_e += labels.size(0)
#         acc_e = correct_e / total_e
        
#         # Sound Val
#         correct_s, total_s = 0, 0
#         with torch.no_grad():
#             for audios, labels in sound_val_loader:
#                 audios, labels = audios.to(DEVICE), labels.to(DEVICE)
#                 logits = audio_model(audios)
#                 pred = logits.argmax(-1)
#                 correct_s += (pred == labels).sum().item()
#                 total_s += labels.size(0)
#         acc_s = correct_s / total_s
        
#         avg_acc = (acc_b + acc_e + acc_s) / 3
        
#         print(f"📊 E{epoch+1}: Behavior {acc_b:.4f}, Emotion {acc_e:.4f}, Sound {acc_s:.4f}, Avg {avg_acc:.4f}")
        
#         history.append({
#             'epoch': epoch+1,
#             'loss_behavior': task_losses['behavior'] / task_counts['behavior'],
#             'loss_emotion': task_losses['emotion'] / task_counts['emotion'],
#             'loss_sound': task_losses['sound'] / task_counts['sound'],
#             'acc_behavior': acc_b,
#             'acc_emotion': acc_e,
#             'acc_sound': acc_s,
#             'acc_avg': avg_acc
#         })
        
#         # 저장
#         if avg_acc > best_avg_acc:
#             best_avg_acc = avg_acc
#             torch.save({
#                 'video_model_state_dict': video_model.state_dict(),
#                 'audio_model_state_dict': audio_model.state_dict(),
#                 'behavior_label_to_id': behavior_train.label_to_id,
#                 'emotion_label_to_id': emotion_train.label_to_id,
#                 'sound_label_to_id': sound_train.label_to_id,
#                 'best_epoch': epoch+1,
#                 'best_acc': best_avg_acc,
#                 'history': history
#             }, 'pet_omni_v2_best.pth')
#             print(f"💾 New best: {best_avg_acc:.4f}")
    
#     # 그래프
#     plt.figure(figsize=(15, 5))
    
#     plt.subplot(131)
#     plt.plot([h['acc_behavior'] for h in history], 'b-', label='Behavior', linewidth=2)
#     plt.legend(); plt.title('Behavior Accuracy'); plt.ylim(0, 1); plt.grid(True, alpha=0.3)
    
#     plt.subplot(132)
#     plt.plot([h['acc_emotion'] for h in history], 'r-', label='Emotion', linewidth=2)
#     plt.legend(); plt.title('Emotion Accuracy'); plt.ylim(0, 1); plt.grid(True, alpha=0.3)
    
#     plt.subplot(133)
#     plt.plot([h['acc_sound'] for h in history], 'g-', label='Sound', linewidth=2)
#     plt.legend(); plt.title('Sound Accuracy'); plt.ylim(0, 1); plt.grid(True, alpha=0.3)
    
#     plt.tight_layout()
#     plt.savefig('pet_omni_v2_history.png', dpi=150, bbox_inches='tight')
#     plt.show()
    
#     print(f"\n🎉 Best Avg Acc: {best_avg_acc:.4f}")

# if __name__ == "__main__":
#     train()